In [81]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', 500)
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

The idea is to use a neural network to predict whether or not a customer would make a purchase, and if yes we want to know the number of campaigns needed per customer.

In [51]:
data_cleaned = pd.read_csv('dataset/cleaned_marketing_campaign.csv', sep=',')

In [52]:
data_cleaned.head()

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Accepted
0,0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,0
1,1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,0
2,2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,0
3,3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,0
4,4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,0


In [69]:
y = data_cleaned['Accepted'].to_numpy()
X = data_cleaned.drop(columns=['Accepted', 'Dt_Customer'])
X = pd.get_dummies(X, columns=['Education','Marital_Status'])
X = X.to_numpy()

In [73]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=10)

for i, (train_index, test_index) in enumerate(split.split(X, y)):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

In [75]:
m,n = X_train.shape

shape_init = tf.keras.Input(shape=(n,))

model = Sequential([shape_init,
    Dense(units=200, activation='relu'),
    Dense(units=500, activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=len(y_train), activation='softmax')],
    name='Accepted_in_which_campaign'
)
model.summary()

Model: "Accepted_in_which_campaign"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 200)               7800      
                                                                 
 dense_38 (Dense)            (None, 500)               100500    
                                                                 
 dense_39 (Dense)            (None, 100)               50100     
                                                                 
 dense_40 (Dense)            (None, 1764)              178164    
                                                                 
Total params: 336,564
Trainable params: 336,564
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(0.001))

In [78]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
56/56 [==============================] - 1s 7ms/step - loss: 208.7286
Epoch 2/30
56/56 [==============================] - 0s 6ms/step - loss: 36.0166
Epoch 3/30
56/56 [==============================] - 0s 6ms/step - loss: 40.0833
Epoch 4/30
56/56 [==============================] - 0s 7ms/step - loss: 18.8215
Epoch 5/30
56/56 [==============================] - 0s 7ms/step - loss: 14.1450
Epoch 6/30
56/56 [==============================] - 0s 8ms/step - loss: 10.6762
Epoch 7/30
56/56 [==============================] - 0s 8ms/step - loss: 9.1114
Epoch 8/30
56/56 [==============================] - 0s 8ms/step - loss: 2.9958
Epoch 9/30
56/56 [==============================] - 0s 7ms/step - loss: 4.7492
Epoch 10/30
56/56 [==============================] - 0s 6ms/step - loss: 2.5869
Epoch 11/30
56/56 [==============================] - 0s 7ms/step - loss: 1.4908
Epoch 12/30
56/56 [==============================] - 0s 8ms/step - loss: 1.4592
Epoch 13/30
56/56 [=======================

In [99]:
y_pred_train = model.predict(X_train)
y_pred_train = np.array([i.argmax() for i in y_pred_train])
y_pred_test = model.predict(X_test)
y_pred_test = np.array([i.argmax() for i in y_pred_test])

14/14 [==============================] - 0s 3ms/step


In [98]:
(y_train == y_pred_train).astype(int).sum()/len(y_train)

0.7919501133786848

In [100]:
(y_test == y_pred_test).astype(int).sum()/len(y_test)

0.7936507936507936